In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import csv
from time import sleep
import requests
import time


def linkedin_extractor(profile_link):
    list_of_name = []
    list_of_experience = []
    list_of_about = []
    list_of_address = []
    list_of_job_title = []
    list_of_description = []
    list_of_skills =[]

    credential = open('C:/Users/kumaramar/OneDrive - Microsoft/hackathon/Auto_Email/credentials.txt')
    line = credential.readlines()
    username = line[0]
    password = line[1]
    credential.close()
#     print('- Finish importing the login credentials')
    sleep(2)

#     t1 = time.time()
#     print('Start-time: ', t1)


    s = Service('C:/Users/kumaramar/OneDrive - Microsoft/hackathon/Auto_Email/chromedriver.exe')
    driver = webdriver.Chrome(service=s)
    driver.get('https://www.linkedin.com')
    sleep(3)
    driver.find_element(By.ID, 'session_key').send_keys(username)
    driver.find_element(By.ID, 'session_password').send_keys(password)
    sleep(2)
    log_in_button = driver.find_element(By.CLASS_NAME, "sign-in-form__submit-button")
    sleep(1)
    log_in_button.click()
    sleep(15)

    try:
        driver.get(profile_link)
        sleep(15)
    #NAME
        name = driver.find_element(By.XPATH, "//h1[@class='text-heading-xlarge inline t-24 v-align-middle break-words']")
#         list_of_name.append(name.text)
        name_text=name.text
        sleep(5)
    except:
        print('Error in profile: ', profile_link)
        list_of_name.append('')
        list_of_experience.append([])
        list_of_about.append([])
        list_of_address.append([''])
        list_of_job_title.append('')
        list_of_description.append([])
        list_of_skills.append([])

    
# TITLE
    title = driver.find_element(By.XPATH, "//div[@class='text-body-medium break-words']")
#     list_of_job_title.append(title.text)
    title_text=title.text
    
# ADDRESS
    address = driver.find_elements(By.XPATH, "//span[@class='text-body-small inline t-black--light break-words']")
    location=[]
    for j in address:
        location.append(j.text)
        list_of_address.append(location)
    

#-----------------------------------BeautifulSoup--------------------------
    src = driver.page_source
    doc = BeautifulSoup(src, 'html.parser')
    div = doc.find_all('section', class_='artdeco-card ember-view relative break-words pb3 mt2')

# ABOUT
    about_section_present= False
    for section in div:
        about_section = section.find_all('div', {'id': 'about'})
        if len(about_section) != 0:
            about_section_present=True
            about_text= section.find_all('span', {'class': 'visually-hidden'})
            about_each=[]
            for bio in about_text:
                about_each.append(bio.text)
            list_of_about.append(about_each)
            break;
    if about_section_present == False: 
        about_each=[]
        list_of_about.append(about_each)
        
# CHECK FOR SKILL AND EXPERIENCE SECTION
    skill_section_present= False
    experience_section_present=False
    
    for section in div:
        skill_section = section.find_all('div', {'id': 'skills'})
        if len(skill_section) != 0:
            skill_section_present=True
            break
    
    for section in div:
        exp_section = section.find_all('div', {'id': 'experience'})
        if len(exp_section) != 0:
            experience_section_present=True
            break
    
    get_url = driver.current_url


# EXPERIENCE

    if experience_section_present==True:
        sleep(3)
        driver.get(get_url+"details/experience")
        sleep(7)
        src2 = driver.page_source
        doc2 = BeautifulSoup(src2, 'html.parser')
        exp_list = doc2.find_all('li', {'class':'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated'})
        
        experience_per_person=[]
        for each_exp in exp_list:
            text_line_list= each_exp.find_all('span', {'class': 'visually-hidden'})
            partition_list=[]
            for text_line in text_line_list:
                partition_list.append(text_line.text)
            experience_per_person.append(partition_list)
        list_of_experience.append(experience_per_person)
        
        

# SKILLS
   
    if skill_section_present==True:
        sleep(3)
        driver.get(get_url+"details/skills")
        driver.maximize_window()

        SCROLL_PAUSE_TIME = 0.8
#-----------------------------------------------
        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

#------------------------------------------------                
        print("Scrolling is over")   
        sleep(5)
        skill_each=[]
        src1 = driver.page_source
        doc1 = BeautifulSoup(src1, 'html.parser')
        div1 = doc1.find_all('li', {'class':'pvs-list__paged-list-item'}) 

        for li in div1:
            only_skill=[]
            skill_text= li.find_all('span', {'class': 'visually-hidden'})
            for skills in skill_text:
                only_skill.append(skills.text)
            skill_each.append(only_skill[0])
        list_of_skills.append(list(set(skill_each)))

    elif skill_section_present == False: 
        skill_each=[]
        list_of_skills.append(skill_each)
        
#     print(list_of_name)
#     print(list_of_job_title) 
#     print(list_of_address)
#     print(list_of_about)
#     print(list_of_experience)
#     for m in list_of_skills:
#         print(len(m),m)
    sleep(8)
    driver.quit()

#     t2 = time.time()
#     print('End-time: ', t2)
#     print('Total time taken: ', (t2-t1)/60, 'minutes\t')

    experience=[]
    for exp in experience_per_person:
        for token in exp:
            experience.append(token)
        experience += ["."]
    experience = " ".join(experience)
    skills= ",".join(skill_each)
    return {
        'name':name_text,
        'profile':title_text +" "+ about_each[1] + " " +  experience + " " + skills}

In [31]:
temp= linkedin_extractor('https://www.linkedin.com/in/reetabratahar/')
temp

Scrolling is over


{'name': 'Reetabrata Har',
 'profile': 'Data Science Lead Experienced in building and leading teams on projects covering the entire spectrum of Data Science and Analytics. In-depth knowledge of Mathematical and Statistical Modeling, Machine Learning, Programming, Data Warehousing and Data Engineering techniques. Background in working for start-ups, global organizations and in academia.  Director, Data and Business Intelligence Coda Payments Jan 2021 - Present · 1 yr 9 mos Singapore . GoBear 5 yrs 7 mos Singapore Director, Technical Product Management Jan 2019 - Dec 2020 · 2 yrs Director Of Analytics Jan 2018 - Dec 2018 · 1 yr Data Scientist Jun 2015 - Dec 2017 · 2 yrs 7 mos . Actuary Data Analyst Aon Jan 2014 - May 2015 · 1 yr 5 mos Singapore . Machine Learning,Statistical Modeling,Data Engineering,Data Science,Cloud Computing,Product Development,Python,R,SQL,Hadoop,Google Cloud Platform (GCP),Amazon Web Services (AWS),Natural Language Processing (NLP),Data Monetization,Apache Spark,Ap

In [ ]:
temp = - Finish importing the login credentials
Start-time:  1663697615.104854
Scrolling is over
End-time:  1663697698.6444213
Total time taken:  1.3923261205355326 minutes	
{'name': 'Reetabrata Har',
 'title': 'Data Science Lead',
 'about': 'Experienced in building and leading teams on projects covering the entire spectrum of Data Science and Analytics. In-depth knowledge of Mathematical and Statistical Modeling, Machine Learning, Programming, Data Warehousing and Data Engineering techniques. Background in working for start-ups, global organizations and in academia. ',
 'experience': 'Director, Data and Business Intelligence Coda Payments Jan 2021 - Present · 1 yr 9 mos Singapore . GoBear 5 yrs 7 mos Singapore Director, Technical Product Management Jan 2019 - Dec 2020 · 2 yrs Director Of Analytics Jan 2018 - Dec 2018 · 1 yr Data Scientist Jun 2015 - Dec 2017 · 2 yrs 7 mos . Actuary Data Analyst Aon Jan 2014 - May 2015 · 1 yr 5 mos Singapore .',
 'skill': 'Machine Learning,Statistical Modeling,Data Engineering,Data Science,Cloud Computing,Product Development,Python,R,SQL,Hadoop,Google Cloud Platform (GCP),Amazon Web Services (AWS),Natural Language Processing (NLP),Data Monetization,Apache Spark,Apache Airflow,Machine Learning,Statistical Modeling,Data Engineering,Data Science,Cloud Computing,Product Development,Natural Language Processing (NLP),Data Monetization,Apache Spark,Python,R,SQL,Hadoop,Google Cloud Platform (GCP),Amazon Web Services (AWS),Apache Airflow'}

In [2]:
# temp={'Name':'Reetabrata Har',
# 'title':'Data Science Lead',
# 'about': ['About', 'Experienced in building and leading teams on projects covering the entire spectrum of Data Science and Analytics. In-depth knowledge of Mathematical and Statistical Modeling, Machine Learning, Programming, Data Warehousing and Data Engineering techniques. Background in working for start-ups, global organizations and in academia. '],
# 'experience':[['Director, Data and Business Intelligence', 'Coda Payments', 'Jan 2021 - Present · 1 yr 9 mos', 'Singapore'], ['GoBear', '5 yrs 7 mos', 'Singapore', 'Director, Technical Product Management', 'Jan 2019 - Dec 2020 · 2 yrs', 'Director Of Analytics', 'Jan 2018 - Dec 2018 · 1 yr', 'Data Scientist', 'Jun 2015 - Dec 2017 · 2 yrs 7 mos'], ['Actuary Data Analyst', 'Aon', 'Jan 2014 - May 2015 · 1 yr 5 mos', 'Singapore']],
# 'skill': ['Apache Spark', 'Machine Learning', 'Data Monetization', 'Google Cloud Platform (GCP)', 'Statistical Modeling', 'Product Development', 'Amazon Web Services (AWS)', 'R', 'SQL', 'Apache Airflow', 'Data Science', 'Hadoop', 'Cloud Computing', 'Data Engineering', 'Natural Language Processing (NLP)', 'Python']}